# RAG Workflow with Reranking

https://docs.llamaindex.ai/en/latest/examples/workflow/rag/

In [1]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.response_synthesizers import CompactAndRefine
from llama_index.core.postprocessor.llm_rerank import LLMRerank
from llama_index.core.workflow import (
    Context,
    Workflow,
    StartEvent,
    StopEvent,
    step,
)

from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding


In [2]:
from llama_index.core.workflow import Event
from llama_index.core.schema import NodeWithScore


class RetrieverEvent(Event):
    """Result of running retrieval"""

    nodes: list[NodeWithScore]


class RerankEvent(Event):
    """Result of running reranking on retrieved nodes"""

    nodes: list[NodeWithScore]

In [3]:
class RAGWorkflow(Workflow):
    @step
    async def ingest(self, ctx: Context, ev: StartEvent) -> StopEvent | None:
        """Entry point to ingest a document, triggered by a StartEvent with `dirname`."""
        dirname = ev.get("dirname")
        if not dirname:
            return None

        documents = SimpleDirectoryReader(dirname).load_data()
        index = VectorStoreIndex.from_documents(
            documents=documents,
            embed_model=OpenAIEmbedding(model_name="text-embedding-3-small"),
        )
        return StopEvent(result=index)

    @step
    async def retrieve(
        self, ctx: Context, ev: StartEvent
    ) -> RetrieverEvent | None:
        "Entry point for RAG, triggered by a StartEvent with `query`."
        query = ev.get("query")
        index = ev.get("index")

        if not query:
            return None

        print(f"Query the database with: {query}")

        # store the query in the global context
        await ctx.set("query", query)

        # get the index from the global context
        if index is None:
            print("Index is empty, load some documents before querying!")
            return None

        retriever = index.as_retriever(similarity_top_k=2)
        nodes = await retriever.aretrieve(query)
        print(f"Retrieved {len(nodes)} nodes.")
        return RetrieverEvent(nodes=nodes)

    @step
    async def rerank(self, ctx: Context, ev: RetrieverEvent) -> RerankEvent:
        # Rerank the nodes
        ranker = LLMRerank(
            choice_batch_size=5, top_n=3, llm=OpenAI(model="gpt-4o-mini")
        )
        print(await ctx.get("query", default=None), flush=True)
        new_nodes = ranker.postprocess_nodes(
            ev.nodes, query_str=await ctx.get("query", default=None)
        )
        print(f"Reranked nodes to {len(new_nodes)}")
        return RerankEvent(nodes=new_nodes)

    @step
    async def synthesize(self, ctx: Context, ev: RerankEvent) -> StopEvent:
        """Return a streaming response using reranked nodes."""
        llm = OpenAI(model="gpt-4o-mini")
        summarizer = CompactAndRefine(llm=llm, streaming=True, verbose=True)
        query = await ctx.get("query", default=None)

        response = await summarizer.asynthesize(query, nodes=ev.nodes)
        return StopEvent(result=response)

In [4]:
w = RAGWorkflow()

# Ingest the documents
index = await w.run(dirname="../data")

In [5]:
# Run a query
result = await w.run(query="Các cunh mệnh nào phù hợp với cơ nghiệp ?", index=index)
async for chunk in result.async_response_gen():
    print(chunk, end="", flush=True)

Query the database with: Các cunh mệnh nào phù hợp với cơ nghiệp ?
Retrieved 2 nodes.
Các cunh mệnh nào phù hợp với cơ nghiệp ?
raw_response: Doc: 1, Relevance: 8  
Doc: 2, Relevance: 5
raw_choices: [1, 2]
relevances: [8.0, 5.0]
Reranked nodes to 2
Các cung mệnh phù hợp với cơ nghiệp bao gồm cung Mệnh an tại Ngọ có sao Kình Dương, vì người có sao này có tác phong quả quyết và sức mạnh kích phát, điều này là một trong những điều kiện cần thiết để sáng lập cơ nghiệp. Ngoài ra, cung Tài bạch an tại Dần có sao Cự Môn cũng có thể phù hợp, đặc biệt khi kết hợp với các tổ hợp khác, cho thấy khả năng làm việc trong các lĩnh vực như giảng dạy, quảng bá hoặc tổ chức sự kiện.

Nghiên cứu tinh mệnh học - Nguyễn Anh Vũ biên soạn
Cung Mệnh an tại Ngọ có sao Kình dương
Có tác phong rất quả quyết và có lực xung kích. Đây là một trong những điều kiện phải có để sáng lập cơ nghiệp.
